<font color='blue' size=5><b>Комментарий ревьюера</b></font>

<font color='blue'>Привет, Леонид! Меня зовут Павел Григорьев, и я буду проверять этот проект.<br>Моя основная цель - не указать на совершённые тобой ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.<br>Спасибо за проделанную работу! Предлагаю общаться на «ты».</font>
<details>
	<summary><u>Инструкция по организационным моментам (кликабельно)</u></summary>
<font color='blue'>Я буду использовать различные цвета, чтобы было удобнее воспринимать мои комментарии:</font>


---


<font color='blue'>синий текст - просто текст комментария</font>

<font color='green'>✔️ и зеленый текст - все отлично</font>

<font color='orange'>⚠️ и оранжевый текст - сделано все правильно, однако есть рекомендации, на что стоит обратить внимание</font>

<font color='red'>❌ и красный текст - есть недочеты</font>


</details>    
    </br>
<font color='blue'>Пожалуйста, не удаляй мои комментарии в случае возврата работы, так будет проще разобраться, какие были недочеты, а также сразу увидеть исправленное. </font>

Ответы на мои комментарии лучше тоже помечать.
Например: <font color='purple'><b>Комментарий студента</b></font>

<font color='blue'><b>Давай смотреть, что получилось!</b></font>


<font color='purple'><b>Комментарий студента</b>   
Павел, привет!   
Спаcибо за ревью.     
У меня нет серьезных вопросов, но может ты сходу ответишь на эти: <br>
* У меня не получилось построить scatter plot для выяснения зависимости population и house_value из-за большого количества данных. Как рекомендуется строить диаграммы по большому количеству точек (самый простой способ)?
* Не получается имортировать некоторые функции из документации pyspark.sql.functions например median(но есть mean), percentile_approx, otherwise не импортируется, но работает после when. Есть какая-то засада, которой я не понимаю... 
* Предупреждения , я прав что
** WARN BLAS: Failed to load implementation from ... - настройки spark servera ?
** WARN Instrumentation: [572dd3d8] regParam is zero, which might cause numerical instability and overfitting. - какие-то значения в данных (в датасете?) подготовкой данных исправить нельзя?
** WARN Instrumentation: [c10784cc] Cholesky solver failed due to singular covariance matriх...  - я понимаю, что это только после OHE, а почему в этом случае не работает Cholesky solver?
  <br>  Если нет ответа, буду искать дальше.
        
Твои замечания поcтарался учесть , эти места прокомментированы и исправлены.    
 </font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>
+ Если я правильно понял проблему, то применяется сэмплирование, для статистики по всему датасету.
+ Да, в этом окружении некоторые функции из модуля pyspark.sql.functions не обнаруживаются.
    + `WARN Instrumentation: regParam is zero, which might cause numerical instability and overfitting` Предупреждение о том, что не используется регуляризация, поэтому модель может переобучаться или работать нестабильно.
    
    + `WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS` Библиотека Basic Linear Algebra Subprograms (BLAS) не подгружена, эта бибилиотека оптимизирует матричные вычисления в Spark ML.
    
    + `WARN Instrumentation: Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.`
Указывает на то, что solver Холецкого дал сбой из-за сингулярной ковариационной матрицы. Это происходит, когда матрица необратима, что означает, что у нее нет обратной матрицы, и поэтому решатель не может разложить ее на нижнюю и верхнюю треугольные матрицы. поэтому применяется более робастый Quasi-Newton solver.</font>

# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

## Подготовка данных

In [1]:
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

ImportError: Can't determine version for bottleneck

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor,LinearRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.mllib.evaluation import RegressionMetrics
from pyspark.ml.feature import StandardScaler#,MinMaxScaler
from pyspark.ml.stat import Correlation

In [ ]:
import matplotlib.pyplot as plt

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Хорошее оформление импортов! </font>

In [ ]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

In [ ]:
# schema = StructType([
#     StructField("longitude", FloatType(), nullable=True),
#     StructField("latitude", FloatType(), nullable=True),
#     StructField("housing_median_age", FloatType(), nullable=True),
#     StructField("total_rooms", FloatType(), nullable=True),
#     StructField("total_bedrooms", FloatType(), nullable=True),
#     StructField("population", FloatType(), nullable=True),
#     StructField("households", FloatType(), nullable=True),
#     StructField("median_income", FloatType(), nullable=True),
#     StructField("median_house_value", FloatType(), nullable=True),
#     StructField("ocean_proximity", StringType(), nullable=True)]
# )

df_housing =  spark.read.csv( '/datasets/housing.csv',inferSchema=True,header=True)
df_housing.printSchema()

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Отлично!  Выведены типы данных.</font>

Прочитанная из датасета схема соответствует ожидаемой

Проверим незаполненные значения

In [ ]:
for c in df_housing.columns:
    print(c,df_housing.select(c).count(),
          df_housing.select(c).filter(F.col(c).isNull() ).count(),
          df_housing.select(c).filter(F.col(c) ==np.nan).count(),
          df_housing.select(c).filter(F.col(c).isin([None]) ).count()
         )

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Обнаружены пропуски!</font>

Только в одном поле есть пустышки, так как их примерно 1% , можно заполнить даже нулями.

In [ ]:
df_housing = df_housing.fillna(0,'total_bedrooms')
df_housing.filter(F.col('total_bedrooms').isNull()).count(),df_housing.filter(F.col('total_bedrooms')==0).count()

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'>Медиану можно найти вот так:</font>
`int(df_housing.approxQuantile('total_bedrooms', [0.5], 0)[0])`

<font color='purple'><b>Комментарий студента</b><br>
Добавил ниже, по-моему лучше заменить на медиану коэффицент (тоже 0), а из него рассчитать количество комнат
</font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> 👍</font>

nulls заменены на 0

Проверим полные дубликаты 

In [ ]:
df_housing.groupby(df_housing.columns[2:]).count().where('count > 1').count()

Проверим корреляцию

In [ ]:
num_cols=[
    'longitude','latitude','housing_median_age','total_rooms','total_bedrooms','population','households','median_income'
]
def corr_to_df(df_rdd,cols):
    return pd.DataFrame(
        Correlation.corr(
                VectorAssembler(inputCols=cols,outputCol='v').transform(df_rdd),'v'
            ).collect()[0][0].toArray().tolist() ,
        index=cols,columns = cols 
    )    

In [ ]:
corr_to_df(df_housing,num_cols).style.background_gradient(cmap='coolwarm').set_precision(2)

Есть несколько колонок , сильно коррелирующих с количеством домов в районе.    
В курсе была идея заменить их на коэффициенты относительно количества домов, что уменьшит прямую корреляцию признаков, но всё же сохранит полезную информацию о плотности населения и количестве комнат на дом, которая может влиять на цену  

Кроме того сильно коррелируют географические координаты, видимо это связано с тем , что Калифорния вытянута c NW на SE.  
Можно повернуть координаты так, чтобы длинная ось была вдоль побережья, а вторая поперек, в этом случае вторая ось будет коррелировать с расстоянием от океана.  
Точный угол поворота можно рассчитать из набора данных (но этом случае он будет зависеть от набора и его надо пересчитывать при предсказании). Я думаю, что достаточно повернуть на 45град, заменив координаты разностью и суммой - это грубое улучшение, но все же лучше, чем исходный набор 

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

In [ ]:
df_housing = df_housing.withColumn('k_rooms', F.col('total_rooms')/F.col('households') )
df_housing = df_housing.withColumn('k_population', F.col('population')/F.col('households') )
df_housing = df_housing.withColumn('k_bedrooms', F.col('total_bedrooms')/F.col('total_rooms') )
df_housing = df_housing.withColumn('sum_xy', F.col('longitude')+F.col('latitude') )
df_housing = df_housing.withColumn('diff_xy', F.col('longitude')-F.col('latitude') )

In [ ]:
new_num_cols = [
   'sum_xy','diff_xy','housing_median_age','k_rooms','k_bedrooms','k_population' ,'households','median_income'
]
corr_to_df(df_housing,new_num_cols).style.background_gradient(cmap='coolwarm').set_precision(2)

В набор трансформированных колонок корреляции гораздо меньше и сильных корреляци нет совсем

<font color='purple'><b>Комментарий студента</b><br>
Здесь можно поправить записи с нулем спален - заменяем нулевой коэффицент на медиану и считаем комнаты 
</font>

In [ ]:
med_bedrooms = df_housing.approxQuantile('k_bedrooms', [0.5], 0)[0]

df_housing=df_housing.withColumn('k_bedrooms',F.when(df_housing.k_bedrooms==0,.2).otherwise(df_housing.k_bedrooms))
df_housing=df_housing.withColumn('total_bedrooms',
        F.when(df_housing.total_bedrooms==0, df_housing.k_bedrooms*df_housing.total_rooms). \
          otherwise(df_housing.total_bedrooms)
        )                         
                                 

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

Наконец , категориальная колонка ocean_proximity видимо монотонна по цене(хотя и нелинейна) и в принципе может быть закодирована числами по возрастанию, так что применение линейной регрессии к ней будет корректно.

In [ ]:
dc = {'ISLAND':4,'NEAR BAY':3,'NEAR OCEAN':2,'<1H OCEAN':1,'INLAND':0}
map_col = F.create_map([F.lit(x) for i in dc.items() for x in i])
df_housing=df_housing.withColumn('i_proxy', map_col[F.col('ocean_proximity')])
num_cols_with_proxy = num_cols + ['i_proxy']
num_cols_with_target = new_num_cols+['i_proxy','median_house_value']
corr_to_df(df_housing,num_cols_with_target).style.background_gradient(cmap='coolwarm').set_precision(2)

Я вывел карту корреляции для модифицированных колонок и цены.  
Видимо, на цену наиболее сильно влияют средний доход в районе  и близость к океану.  
(Также можно отметить корреляцию между i_proxy и суммой координат - положением по оси SW-NE - обе показывают расстояние от океана)    
Неожиданно цена умеренно антикоррелирует с количеством спален.  
Влияние остальных признаков видимо невелико.

Все подготовительные преобразования не использовали аггрегированные свойства датасетов и основывались только на переменных из одной записи. 
Поэтому их корректно применить ко всем тестовым и обучающим наборам до их разделения и начала обучения.

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

## Обучение моделей

Создадим две пары обучающих/тестовых датасетов - для теста трансформаций признаков и для финального теста

In [ ]:
(trfi,te ) = df_housing.randomSplit([0.75, 0.25],seed=499)
(tr,  va ) = trfi.randomSplit([0.75, 0.25],seed=4999)

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> Здорово, что у нас есть три выборки!</font>

Класс для прогона датасетов через пайплайн.  
В двух вариантах - c OHE трансформацией и без неё на числовых признаках, различаются только построением пайплайна
После каждого прогона вычисляются метрики и запсываются в атрибут rm

In [ ]:
class ExecPlan:
    def __init__(self,name,num_cols):
        self.name = name
        self._id = ''.join([c for c in name if c.isupper()])
        self.num_cols = num_cols
        self.stages = []
        self.ta_col = 'median_house_value'
        self.cat_col = 'ocean_proximity'
        
    def create_pipe(self): 
        self.pipe = Pipeline(stages = self.stages)

        
class NumOnlyPlan(ExecPlan):
        
    def create_pipe(self):
        self.stages.append( VectorAssembler( inputCols = self.num_cols ,outputCol='va_'+self._id))
        self.stages.append( StandardScaler( inputCol = 'va_'+self._id ,outputCol='fe_'+self._id, withMean=True))
        self.stages.append( LinearRegression(featuresCol='fe_'+self._id, 
                                             labelCol=self.ta_col,
                                             predictionCol='pr_'+self._id))
        super().create_pipe()
           
    
class OhePlan(ExecPlan):    
        
    def create_pipe(self):
        self.stages.append(StringIndexer(inputCol=self.cat_col ,outputCol='si_'+self._id,handleInvalid='keep'))
        self.stages.append(OneHotEncoder(inputCol='si_'+self._id ,outputCol='ohe_'+self._id))                   
        self.stages.append(VectorAssembler(inputCols=self.num_cols+['ohe_'+self._id] ,outputCol='va_'+self._id))
        self.stages.append( StandardScaler( inputCol = 'va_'+self._id ,outputCol='fe_'+self._id, withMean=True))
        self.stages.append( LinearRegression(featuresCol='fe_'+self._id, 
                                             labelCol=self.ta_col,
                                             predictionCol='pr_'+self._id))
        super().create_pipe()
    
 

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='blue'>Бывает так, что в тестовых данных случается категория которой нет в обучающих. В StringIndexer есть три стратегии обработки незнакомых Лейблов.
1. Выбрасывать исключение (по умолчанию).
2. Игнорировать строки с незнакомыми лейблами.
3. Выделять для всех незнакомых лейблов отдельную категорию.

За это отвечает аргумент [handleInvalid](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StringIndexer.html#pyspark.ml.feature.StringIndexer.handleInvalid), в который нужно передать "error","skip" или "keep" соответственно.</font>

<font color='purple'><b>Комментарий студента</b><br>
Т.е нормально все же должно быть keep, если доп.значения не фатальны?
</font>

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>Варьируетсся от количества редких категорий и возможном появлении новых в будущем.</font>

Проверим на тестовом наборе элементарные трансформации (по 1-2 колонки) - что мы от них приобретаем и теряем.   
Первая конфигурация - исходные числовые колонки без OHE рассматривается как эталон, с ней будем сравнивать остальные.

In [ ]:
plans= (NumOnlyPlan(name='Original num columns ____',num_cols=num_cols),
        NumOnlyPlan(name='geo XY rotated___________',num_cols=['sum_xy','diff_xy']+num_cols[2:]),
        NumOnlyPlan(name='Population transformed___',num_cols=['k_population' if x=='population'else x for x in num_cols]), 
        NumOnlyPlan(name='Rooms&Bedrooms transformed',num_cols=num_cols[:3]+['k_rooms','k_bedrooms']+num_cols[5:]), 
        NumOnlyPlan(name='Ocean_Proxy as Ordinal ___', num_cols=num_cols+['i_proxy']),
        OhePlan( name =  'OHE transformation _______' ,num_cols=num_cols)
       ) 


for p in plans:
    p.create_pipe()
    p.pipe= p.pipe.fit(tr)
    va= p.pipe.transform(va)
    pred_col = 'pr_'+p._id
    p.rm= RegressionMetrics( va.rdd.map(lambda r: ( r[pred_col],r.median_house_value ) ) )

## Анализ результатов

In [ ]:
for p in plans:
    p.out = f"{p.name}\t{round(p.rm.r2,4)}\t{round(p.rm.rootMeanSquaredError)}\t{round(p.rm.meanAbsoluteError)}"


Выведем результат в виде таблицы

In [ ]:
print(f"____Тип трансформации____\t __R2__\t_RMSE_\t_MAE_")
for p in plans:
    print(p.out)

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

Но может быть более наглядно будет графически:

In [ ]:
res=pd.DataFrame(list(map(lambda e:[e.name,e.rm.r2 ],plans)),columns=['name','R2']).sort_values(by='R2').set_index('name')

In [ ]:
def plot_datasets_diff(df,base,title='Эффект трансформаций данных'):
    ax = df.plot.barh(alpha=.6,figsize=(24,6),title=title)
    ax.barh(df.index,df.R2-base ,color='r',height = 0.5 ,alpha=.6,label='diff with orig')
    ax.set_xlabel('R2')
    ax.set_ylabel('Transformations')
    ax.legend(['R2 test res','diff with orig'])
plot_datasets_diff(res,res.loc['Original num columns ____','R2'])    

* OHE категориальной колонки улучшает метрики
* Порядковое кодирование  категориальной колонки также улучшает метрики, но гораздо слабее
* Также улучшает метрики  переход к коэффициентам на дом для количества комнат и спален
* Переход о населения района к заселенности дома неожиданно ухудшает метрику

Последний пункт я объяснить не могу, возможно зависимость цены от заселенности сильно нелинейна


<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

## ФИНАЛЬНЫЙ ТЕСТ

Для финального теста взяты признаки из трансформировнного набора колонок , кроме k_population, так он по непонятной причине портит метрику  

Так же не используется порядковое пробразование ocean_proximity -> i_proxy, так как я ожидаю, что его полезный эффект все равно будет перекрыт OHE.

In [ ]:
fin = OhePlan(name='Final test' ,
    num_cols=['sum_xy','diff_xy','housing_median_age','k_rooms','k_bedrooms','population','households','median_income'])
fin.create_pipe()
fin.pipe= fin.pipe.fit(trfi)
te= fin.pipe.transform(te)
pred_col = 'pr_'+fin._id
fin.rm= RegressionMetrics( te.rdd.map(lambda r: ( r[pred_col],r.median_house_value ) ) )

Метрики теста

In [ ]:
print( "R2 ____", round(fin.rm.r2,4))
print( "RMSE __", round(fin.rm.rootMeanSquaredError,0))
print( "MAE ___", round(fin.rm.meanAbsoluteError,0))

<font color='purple'><b>Комментарий студента</b><br>
Основная метрика улучшилась на 0.0018 по сравнению с версией 1.0
</font>

<font color='blue'><b>Комментарий ревьюера :</b></font> ✔️\
<font color='green'>Тестирование проведено верно.</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'>Не забывай останавливать сессию `spark.stop()`</font>

<font color='purple'><b>Комментарий студента</b><br>
Sorry
</font>

In [ ]:
spark.stop()

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>Здорово, что не забываешь останавливать сессию!</font>

## Выводы

* Построенная модель позволяет предсказывать медианную цену дома в районе с метрикой R2=.649
* На цену дома влияет средний доход в районе, близость к океану и в меньшей степени - отношение колчества спален ко всем комнатам

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Очень приятно видеть вывод в конце проекта!\
Приведены ответы на главные вопросы проекта.</font>

<font color='blue'><b>Итоговый комментарий ревьюера </b></font> \
<font color='green'>Леонид, хороший проект получился! Большое спасибо за проделанную работу. Видно, что приложено много усилий. Полностью описан ход решения, даны все необходимые пояснения, выводы полные и логичные.

<font color='green'><b>Работа уже может быть зачтена</b>, отправляю обратно, чтобы была возможность задать вопросы если они есть  ))).</font>

<font color='blue'><b>Итоговый комментарий ревьюера 2</b></font>\
<font color='green'> Леонид, проект принят! \
Все этапы пройдены. Все рекомендации учтены.\
Надеюсь, тебе понравился процесс выполнения и результат.</font> \
<font color='blue'><b>Спасибо, удачи в освоении профессии!</b></font>

<font color='green'>Немного полезного материала:
+  https://sparkbyexamples.com/pyspark-tutorial/, https://sparkbyexamples.com/
+  https://github.com/dvgodoy/handyspark
+  https://www.tutorialspoint.com/pyspark/index.htm
+  https://www.guru99.com/pyspark-tutorial.html
+  https://databricks.com/spark/getting-started-with-apache-spark/machine-learning#load-sample-data
</font>

In [ ]:
SparkContext(conf=conf )